In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("loan_approval_data.csv")

In [ ]:
df.head()
df.info()
df.isnull().sum()
df.describe()

# Handle Missing Values

In [ ]:
categorical_cols = df.select_dtypes(include = ["object"]).columns
numerical_cols = df.select_dtypes(include = ["number"]).columns

In [ ]:
categorical_cols.size + numerical_cols.size

In [ ]:
from sklearn.impute import SimpleImputer

num_imp = SimpleImputer(strategy = "mean")
df[numerical_cols] = num_imp.fit_transform(df[numerical_cols])


In [ ]:
cat_imp = SimpleImputer(strategy = "most_frequent")
df[categorical_cols] = cat_imp.fit_transform(df[categorical_cols])

In [ ]:
df = df.drop("Applicant_ID", axis = 1)
df.head()
df.isnull().sum()

# EDA

In [ ]:
# How balanced our classes are?

classes_count = df["Loan_Approved"].value_counts()
plt.pie(classes_count, labels = ["No", "Yes"], autopct = "%1.1f%%")
plt.title("IS Loan Approved or not?")

In [ ]:
# Analyze Categories
gender_cnt = df["Gender"].value_counts()
ax = sns.barplot(gender_cnt)
ax.bar_label(ax.containers[0])

In [ ]:
edu_cnt = df["Education_Level"].value_counts()
ax = sns.barplot(edu_cnt)
ax.bar_label(ax.containers[0])

In [ ]:
# Analyze Income

sns.histplot(
    data = df,
    x = "Applicant_Income",
    bins = 20
)

In [ ]:
# Analyze Income

sns.histplot(
    data = df,
    x = "Coapplicant_Income",
    bins = 20
)

In [ ]:
# Outliers - box plots

sns.boxplot(
    data = df,
    x = "Loan_Approved",
    y = "Applicant_Income"
)

In [ ]:
fig, axes = plt.subplots(2,2)
sns.boxplot(ax = axes[0, 0], data = df, x = "Loan_Approved", y = "Applicant_Income")
sns.boxplot(ax = axes[0, 1], data = df, x = "Loan_Approved", y = "Credit_Score")
sns.boxplot(ax = axes[1, 0], data = df, x = "Loan_Approved", y = "DTI_Ratio")
sns.boxplot(ax = axes[1, 1], data = df, x = "Loan_Approved", y = "Savings")
plt.tight_layout()

In [ ]:
# Credit Score with Loan Apporval

sns.histplot(
    data = df,
    x = "Credit_Score",
    bins = 20,
    hue = "Loan_Approved",
    multiple = "dodge"
)

In [ ]:
# Credit Score with Loan Apporval

sns.histplot(
    data = df,
    x = "Applicant_Income",
    bins = 20,
    hue = "Loan_Approved",
    multiple = "dodge"
)

In [ ]:
df.head()


# Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# LabelEncoding
le = LabelEncoder()
df["Education_Level"] = le.fit_transform(df["Education_Level"])
df["Loan_Approved"] = le.fit_transform(df["Loan_Approved"])

In [ ]:
df.head()

In [ ]:
#One Hot Encoding

cols = ["Employment_Status", "Marital_Status", "Loan_Purpose", "Property_Area", "Gender", "Employer_Category"]

ohe = OneHotEncoder(drop = "first", sparse_output = False, handle_unknown = "ignore")

encoded = ohe.fit_transform(df[cols])

encoded_df = pd.DataFrame(encoded, columns= ohe.get_feature_names_out(cols), index = df.index )

In [ ]:
ohe.get_feature_names_out(cols)

In [ ]:
df = pd.concat([df.drop(columns = cols), encoded_df], axis = 1)

In [ ]:
df.head()
df.info()

In [ ]:
df.columns

# Correlation Heatmap


In [ ]:
num_cols = df.select_dtypes(include = "number")
corr_mat = num_cols.corr()

In [ ]:
plt.figure(figsize = (15,8))
sns.heatmap(
    corr_mat,
    annot = True,
    fmt = ".2f",
    cmap = "coolwarm"
)

# Train-Test-Split + Feature Scaling

In [ ]:
X = df.drop("Loan_Approved", axis = 1)
y = df["Loan_Approved"]

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2,
    random_state = 42
)

In [ ]:
X_train.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_test_scaled

# Train and Evaluate Models

In [ ]:
#Logistic Regression

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

log_model = LogisticRegression()
log_model.fit(X_train_scaled, y_train)

y_pred = log_model.predict(X_test_scaled)

In [ ]:
# Evaluation

print("Logistic Regression Model")
print("Precision: ", precision_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))
print("F1 score: ", f1_score(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Confusion Matrix: ", confusion_matrix(y_test, y_pred))

In [ ]:
#KNN

from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors = 5)
knn_model.fit(X_train_scaled, y_train)

y_pred = knn_model.predict(X_test_scaled)

# Evaluation

print("KNN Model")
print("Precision: ", precision_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))
print("F1 score: ", f1_score(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Confusion Matrix: ", confusion_matrix(y_test, y_pred))

In [ ]:
# Naive Bayes

from sklearn.naive_bayes import GaussianNB

nb_model = GaussianNB()
nb_model.fit(X_train_scaled, y_train)

y_pred = nb_model.predict(X_test_scaled)

# Evaluation

print("Naive Bayes Model")
print("Precision: ", precision_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))
print("F1 score: ", f1_score(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Confusion Matrix: ", confusion_matrix(y_test, y_pred))